In [1]:
import os
from datetime import datetime
import logging

import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
from simpletransformers.classification import ClassificationModel, ClassificationArgs
#from simpletransformers.t5 import T5Model, T5Args
import warnings
import pandas as pd
import os
from datetime import datetime
import logging
import glob
from pathlib import Path
import csv

In [2]:
import spacy
from spacy.language import Language
from spacy import displacy
import time

@Language.component("newsent")
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        #print(token.text, token.text in ("’s", "'s"))
        if token.text.upper() in (";", "--", "\n\n", "\n", "QUARTERLY", "STORY", "\n\n\n\n", "\n\n\n"):
            #print("Detected:", token.text)
            doc[token.i].is_sent_start = True
    return doc

#spacy.require_gpu()
nlp = spacy.load("../../Summary/NER/RelateEntity/train/model-best-local")
nlp.add_pipe('sentencizer')
nlp.add_pipe('newsent', name="newsent", last=True)

c:\users\ankan\appdata\local\programs\python\python37\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
c:\users\ankan\appdata\local\programs\python\python37\lib\site-packages\spacy_transformers\pipeline_component.py:406: UserWarning: Automatically converting a transformer component from spacy-transformers v1.0 to v1.1+. If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spacy-transformers version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


<function __main__.set_custom_boundaries(doc)>

In [3]:
def getSentences(inputfile, nlp, text=None):
    if(not text):
        with open(inputfile, 'r', encoding="utf-8") as f:
            text = f.read()

    doc = nlp(text)
    sentences = [str(sent).strip() for sent in doc.sents]

    #print(len(sentences))
    return(sentences)

In [4]:
def clean_unnecessary_spaces(out_string):
    if not isinstance(out_string, str):
        warnings.warn(f">>> {out_string} <<< is not a string.")
        out_string = str(out_string)
    out_string = (
        out_string.replace(" .", ".")
        .replace(" ?", "?")
        .replace(" !", "!")
        .replace(" ,", ",")
        .replace(" ' ", "'")
        .replace(" n't", "n't")
        .replace(" 'm", "'m")
        .replace(" 's", "'s")
        .replace(" 've", "'ve")
        .replace(" 're", "'re")
    )
    return out_string

In [5]:
import hashlib
def dedupList(flist):
    hashList = list()
    newList = list()
    for item in flist:
        #print(item.upper())
        hash_object = hashlib.sha1(item.upper().encode("UTF-8"))
        hex_dig = hash_object.hexdigest()
        if hex_dig not in hashList:
            hashList.append(hex_dig)
            #print(item, hex_dig)
            newList.append(item)
    return(newList)

In [6]:
def isPhrasingRqd(pfile):
    with open(pfile, 'r', encoding = "utf-8") as fp:
        for l_no, line in enumerate(fp):
            if "NOPAD***" in line:
                return False
    return True

In [7]:
devDir = "../../Summary/DATA/PARAPHRASE/Dev"
trainDir = "../../Summary/DATA/PARAPHRASE/Train"
testDir = "../../Summary/DATA/PARAPHRASE/Test"

spdevDir = "../../Summary/DATA/SENTPAIR/Dev"
sptrainDir = "../../Summary/DATA/SENTPAIR/Train"
sptestDir = "../../Summary/DATA/SENTPAIR/Test"

rplStr = ["PG*** ", "ED*** ", "SCHQ*** ", "SCBQ*** ", "SCBF*** ", "SCHF*** ", "SCG*** ", "GF*** ", "GQ*** ", "SC*** ", "NOPAD*** "]

def preProcessSent(line):
    for s in rplStr:
        line = line.replace(s, "")
    if("TBLST***" in line or "TBLET***" in line or "CS***" in line or "@@@" in line or line == "\n" or line == "\n\n" or "https://finance.yahoo.com" in line):
        return(line, False)
    line = line.replace("\n", "")
    if(line == None or line == ""):
        return(line, False)
    line = line.replace("$(0.", "$(.")
    line = line.replace("($0.", "($.")
    line = line.replace("$0.", "$.")
    line = line.replace("$ 0.", "$.")
    return(line, True)

def createTrainingData(inputDir, ppModel=None, writetofile=False):
    files = glob.glob(inputDir+"/*_EP_YH.txt")
    #print(files)
    #print(inputDir)
    if(len(files) > 0):
        for file in (files):
            print("Input file " + file)
            basefile = os.path.basename(file)
            inputfile = os.path.splitext(basefile)[0]
            #print(inputfile)
            if not ppModel:
                outfilePath = inputfile + "_phrase.tsv"
                outfilePath = inputDir + "/" + outfilePath
                print("Phrase file " + outfilePath)
                outfile = Path(outfilePath)
                if outfile.is_file():
                    print("Phrase file " + str(outfile) + " already exists")
                    continue
                with open(outfile, "w", encoding = "utf-8") as of:
                    #cnt = 0
                    of.write("filename\tSentence1\tSentence2\n")
                    #with open(file, encoding="utf-8") as f:
                    sentences = getSentences(file, nlp)
                    #line = f.readline()
                    for line in sentences:
                        for s in rplStr:
                            line = line.replace(s, "")
                        if("TBLST***" in line or "TBLET***" in line or "CS***" in line or "@@@" in line or line == "\n" or line == "\n\n" or "https://finance.yahoo.com" in line):
                            #line = f.readline()
                            continue
                        line = line.replace("\n", "")
                        if(line == ""):
                            continue
                        #print(line)
                        #cnt = cnt + 1
                        phraseLine = line
                        of.write(inputfile+"\t"+line+"\t"+line+"\n")
                        #of.write(line + "\n")
                        #line = f.readline()
            else:
                f = None
                if(writetofile):
                    basefile = os.path.basename(file)
                    inputfile = os.path.splitext(basefile)[0]
                    print("Input file " + inputfile)
                    print(inputDir)
                    outfilePath = inputfile + "_sp.tsv"
                    outfilePath = inputDir + "/" + outfilePath
                    print("SP file " + outfilePath)
                    outfile = Path(outfilePath)
                    if outfile.is_file():
                        print("SP file " + str(outfile) + " already exists")
                        continue
                    f = open(outfile, "w", encoding = "utf-8")
                    f.write("filename\tSentence1\tSentence2\tTarget\n")
                sentences = getSentences(file, nlp)
                #line = f.readline()
                for line in sentences:
                    for s in rplStr:
                        line = line.replace(s, "")
                    if("TBLST***" in line or "TBLET***" in line or "CS***" in line or "@@@" in line or line == "\n" or line == "\n\n" or "https://finance.yahoo.com" in line):
                        #line = f.readline()
                        continue
                    line = line.replace("\n", "")
                    if(line == None or line == ""):
                        continue
                    #print(line)
                    #cnt = cnt + 1
                    #phraseLine = line
                    line = line.replace("$(0.", "$(.")
                    line = line.replace("($0.", "($.")
                    line = line.replace("$0.", "$.")
                    line = line.replace("$ 0.", "$.")
                    inp = [line]
                    #print(inp)
                    if(not writetofile):
                        print("ORIGINAL SENTENCE\n",line)
                        print("PREDICTED PARAPHRASE\n")
                        predicted = ppModel.predict(inp)
                        print(predicted)
                        print("\n")
                    else:
                        predicted = ppModel.predict(inp)
                        if(predicted):
                            predicted = dedupList(predicted[0])
                            #print("ORIGINAL SENTENCE\n",line)
                            #print("PREDICTED PARAPHRASE\n")
                            #print(predicted)
                            for pred in predicted:
                                if(f):
                                    f.write(inputfile+"\t"+line+"\t"+pred+"\t"+"1"+"\n")
                                    print(inputfile+"\t"+line+"\t"+pred+"\t"+"1")
                            #print("\n")
                if(f):
                    f.close()

In [8]:
def predictPhrase(line, ppModel, spModel, logf=None):
    nline, isProcess = preProcessSent(line)
    if not isProcess:
        return(nline)
    inp = [nline]
    print("\n\nORIGINAL LINE:\n", line)
    if(logf):
        logf.write("\n\nORIGINAL LINE:\n" + line)
    predicted = ppModel.predict(inp)
    if(predicted):
        predicted = dedupList(predicted[0])
        print("PREDICTED PARAPHRASE:\n", predicted)
        if(logf):
            logf.write("\nPREDICTED PARAPHRASE:\n" + str(predicted))
        for pred in predicted:
            spinp = [[nline, pred]]
            #print(spinp)
            spred, output = spModel.predict(spinp)
            print("PREDICTED SENTENCE PAIR:\n", spinp, spred)
            if(logf):
                logf.write("\nPREDICTED SENTENCE PAIR:\n" + str(spinp) + str(spred))
            if(spred[0] == 1):
                #print(spinp, spred)
                if(pred[0] == " "):
                    pred = pred[1:]
                print("PREDICTED SENTENCE:\n", pred)
                if(logf):
                    logf.write("\nPREDICTED SENTENCE:\n" + str(pred))
                return(pred)
    print("PREDICTED SENTENCE:\n", nline)
    if(logf):
        logf.write("\nPREDICTED SENTENCE:\n" + nline)
    return(nline)

In [9]:
devDataFile = "../../Summary/DATA/PARAPHRASE/Dev/dev.tsv"
trainDataFile = "../../Summary/DATA/PARAPHRASE/Train/train.tsv"

devSPFile = "../../Summary/DATA/SENTPAIR/Dev/dev.tsv"
trainSPFile = "../../Summary/DATA/SENTPAIR/Train/train.tsv"

def writeTrainingData(writeFile, writeDir):
    if("PARAPHRASE" in writeDir):
        files = glob.glob(writeDir+"/*_phrase.tsv")
    else:
        files = glob.glob(writeDir+"/*_sp.tsv")
    print(files)
    frames = list()

    if(len(files) > 0):
        for file in files:
            df = pd.read_csv(file, sep="\t", encoding = "utf-8").astype(str)
            df = df.dropna()
            df = df[df['Sentence1'].notna()]
            #print(df)
            frames.append(df)
    result = pd.concat(frames)
    print(result)
    result.to_csv(writeFile, sep='\t', index=False, header=True)

In [11]:
createTrainingData(devDir)
writeTrainingData(devDataFile, devDir)

Input file ../../Summary/DATA/PARAPHRASE/Dev\APPN_2023-02-16_EP_YH.txt
Phrase file ../../Summary/DATA/PARAPHRASE/Dev/APPN_2023-02-16_EP_YH_phrase.tsv
Phrase file ..\..\Summary\DATA\PARAPHRASE\Dev\APPN_2023-02-16_EP_YH_phrase.tsv already exists
Input file ../../Summary/DATA/PARAPHRASE/Dev\BILL_2023-02-02_EP_YH.txt
Phrase file ../../Summary/DATA/PARAPHRASE/Dev/BILL_2023-02-02_EP_YH_phrase.tsv
Phrase file ..\..\Summary\DATA\PARAPHRASE\Dev\BILL_2023-02-02_EP_YH_phrase.tsv already exists
Input file ../../Summary/DATA/PARAPHRASE/Dev\CFLT_2022-11-02_EP_YH.txt
Phrase file ../../Summary/DATA/PARAPHRASE/Dev/CFLT_2022-11-02_EP_YH_phrase.tsv
Phrase file ..\..\Summary\DATA\PARAPHRASE\Dev\CFLT_2022-11-02_EP_YH_phrase.tsv already exists
Input file ../../Summary/DATA/PARAPHRASE/Dev\CRWD_2022-11-29_EP_YH.txt
Phrase file ../../Summary/DATA/PARAPHRASE/Dev/CRWD_2022-11-29_EP_YH_phrase.tsv
Phrase file ..\..\Summary\DATA\PARAPHRASE\Dev\CRWD_2022-11-29_EP_YH_phrase.tsv already exists
Input file ../../Summary

In [12]:
createTrainingData(trainDir)
writeTrainingData(trainDataFile, trainDir)

Input file ../../Summary/DATA/PARAPHRASE/Train\APPN_2022-05-06_EP_YH.txt
Phrase file ../../Summary/DATA/PARAPHRASE/Train/APPN_2022-05-06_EP_YH_phrase.tsv
Phrase file ..\..\Summary\DATA\PARAPHRASE\Train\APPN_2022-05-06_EP_YH_phrase.tsv already exists
Input file ../../Summary/DATA/PARAPHRASE/Train\APPN_2022-08-04_EP_YH.txt
Phrase file ../../Summary/DATA/PARAPHRASE/Train/APPN_2022-08-04_EP_YH_phrase.tsv
Phrase file ..\..\Summary\DATA\PARAPHRASE\Train\APPN_2022-08-04_EP_YH_phrase.tsv already exists
Input file ../../Summary/DATA/PARAPHRASE/Train\APPN_2022-11-03_EP_YH.txt
Phrase file ../../Summary/DATA/PARAPHRASE/Train/APPN_2022-11-03_EP_YH_phrase.tsv
Phrase file ..\..\Summary\DATA\PARAPHRASE\Train\APPN_2022-11-03_EP_YH_phrase.tsv already exists
Input file ../../Summary/DATA/PARAPHRASE/Train\APPN_2023-05-09_EP_YH.txt
Phrase file ../../Summary/DATA/PARAPHRASE/Train/APPN_2023-05-09_EP_YH_phrase.tsv
Phrase file ..\..\Summary\DATA\PARAPHRASE\Train\APPN_2023-05-09_EP_YH_phrase.tsv already exists


                 filename                                          Sentence1  \
0   APPN_2022-05-06_EP_YH                                Appian Corporation.   
1   APPN_2022-05-06_EP_YH   First quarter cloud subscription revenue incr...   
2   APPN_2022-05-06_EP_YH         MCLEAN, Va., May 05, 2022 (GLOBE NEWSWIRE)   
3   APPN_2022-05-06_EP_YH  -- Appian (Nasdaq: APPN) today announced finan...   
4   APPN_2022-05-06_EP_YH   “We exceeded guidance and grew cloud subscrip...   
..                    ...                                                ...   
67    ZS_2023-03-03_EP_YH  Accordingly, we are required to add back the n...   
68    ZS_2023-03-03_EP_YH  Additionally, we include the anti-dilutive imp...   
69    ZS_2023-03-03_EP_YH  We have not reconciled our expectations to non...   
70    ZS_2023-03-03_EP_YH  For those reasons, we are also unable to addre...   
71    ZS_2023-03-03_EP_YH  Accordingly, a reconciliation for the guidance...   

                                       

In [12]:
train_df = pd.read_csv(trainDataFile, sep="\t", encoding = "utf-8").astype(str)
eval_df = pd.read_csv(devDataFile, sep="\t", encoding = "utf-8").astype(str)

train_df = train_df.rename(
    columns={"Sentence1": "input_text", "Sentence2": "target_text"}
)
eval_df = eval_df.rename(
    columns={"Sentence1": "input_text", "Sentence2": "target_text"}
)

train_df = train_df[["input_text", "target_text"]]
eval_df = eval_df[["input_text", "target_text"]]

train_df["prefix"] = "paraphrase"
train_df = train_df[["prefix", "input_text", "target_text"]]

eval_df["prefix"] = "paraphrase"
eval_df = eval_df[["prefix", "input_text", "target_text"]]

train_df = train_df.dropna()
train_df = train_df[train_df['input_text'].notna()]

eval_df = eval_df.dropna()
eval_df = eval_df[eval_df['input_text'].notna()]

train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)
print("TRAIN DATA ..............")
print(train_df)

eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)
print("EVAL DATA ..............")
print(eval_df)

TRAIN DATA ..............
          prefix                                         input_text  \
0     paraphrase                                Appian Corporation.   
1     paraphrase   First quarter cloud subscription revenue incr...   
2     paraphrase         MCLEAN, Va., May 05, 2022 (GLOBE NEWSWIRE)   
3     paraphrase  -- Appian (Nasdaq: APPN) today announced finan...   
4     paraphrase   “We exceeded guidance and grew cloud subscrip...   
...          ...                                                ...   
2085  paraphrase  Accordingly, we are required to add back the n...   
2086  paraphrase  Additionally, we include the anti-dilutive imp...   
2087  paraphrase  We have not reconciled our expectations to non...   
2088  paraphrase  For those reasons, we are also unable to addre...   
2089  paraphrase  Accordingly, a reconciliation for the guidance...   

                                            target_text  
0                                   Appian Corporation.  
1    

In [10]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

In [11]:
def count_matches(labels, preds):
    print(labels)
    print(preds)
    return sum(
        [
            1 if label == pred else 0
            for label, pred in zip(labels, preds)
        ]
    )


In [12]:
model_args = Seq2SeqArgs()
#model_args = T5Args()
model_args.do_sample = True
model_args.train_batch_size = 4
model_args.use_multiprocessing = False
model_args.num_train_epochs = 2
#model_args.learning_rate = 5e-5
#model_args.no_save = True

model_args.eval_batch_size = 4
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.evaluate_during_training_steps = 50

model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = False
model_args.save_steps = -1

model_args.max_length = 64
#model_args.max_seq_length = 32
model_args.num_return_sequences = 10
model_args.top_k = 50
model_args.top_p = 0.95
model_args.fp16 = False
#model_args.num_beams = None

model_args.use_early_stopping = False
#model_args.early_stopping_delta = 0.01
#model_args.early_stopping_metric = "mcc"
#model_args.early_stopping_metric_minimize = False
#model_args.early_stopping_patience = 5
#model_args.evaluate_during_training_steps = 500
#model_args.wandb_project =  "visualization-demo"

model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-base",
    args=model_args,
    use_cuda=True,
)

#model = T5Model("t5", "t5-base", args=model_args, use_cuda=True)

# Train the model
model.train_model(
    train_df, eval_data=eval_df, matches=count_matches
)
results = model.eval_model(eval_df)

KeyboardInterrupt: 

In [13]:
model_args = Seq2SeqArgs()
#model_args = T5Args()
model_args.do_sample = True
#model_args.dataset_class = False
#model_args.train_batch_size = 4
model_args.train_batch_size = 8
model_args.use_multiprocessing = False
model_args.num_train_epochs = 2
#model_args.learning_rate = 5e-5
#model_args.no_save = True

#model_args.eval_batch_size = 4
model_args.eval_batch_size = 16
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.evaluate_during_training_steps = 50

model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = False
model_args.save_steps = -1

model_args.max_length = 64
#model_args.max_seq_length = 32
model_args.num_return_sequences = 10
#model_args.top_k = 50
model_args.top_k = 30
#model_args.top_p = 0.95
model_args.top_p = 0.97
model_args.fp16 = False
#model_args.num_beams = None

model_args.use_early_stopping = False
#model_args.early_stopping_delta = 0.01
#model_args.early_stopping_metric = "mcc"
#model_args.early_stopping_metric_minimize = False
#model_args.early_stopping_patience = 5
#model_args.evaluate_during_training_steps = 500
model_args.wandb_project =  "visualization-demo"

In [14]:
model = Seq2SeqModel(
    encoder_decoder_type="bart", encoder_decoder_name="outputs", args=model_args
)

print(
    model.predict(
        [
            "Non-GAAP gross profit was $225.4 million, representing an 86.7% non-GAAP gross margin, compared to $133.5 million, or a 85.3% non-GAAP gross margin, in the second quarter of fiscal 2022"
        ]
    )
)

outputs\model_args.json


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

[['Non-GAAP gross profit is $225.4 million and non-GAap gross margin is 86.7%.', 'Non-GAAP gross profit is $225.4 million and non-GAap gross margin is 86.7%.', 'Non-GAAP gross profit is $225.4 million.', 'Non-GAAP gross profit is $225.4 million and non-GAap gross margin is 86.7%.', 'Non-GAAP gross profit is $225.4 million and non-GAap gross margin is 86.7%.', 'Non-GAAP gross profit is $225.4 million and non-GAap gross margin is 86.7%.', 'Non-GAAP gross profit is $225.4 million and non-GAap gross margin is 86.7%.', 'Non-GAAP gross profit is $225.4 million and non-GAap gross margin is 86.7%.', 'Non-GAAP gross profit is $225.4 million, and non-GAap gross margin is 86.7%.', 'Non-GAAP gross profit is $225.4 million and non-GAap gross margin is 86.7%.']]


In [10]:
createTrainingData(testDir, model)

Input file ../../Summary/DATA/PARAPHRASE/Test\ZS_2022-05-26_EP_YH.txt
Detected: 

Detected: 

Detected: 

Detected: 

Detected: 



Detected: 

Detected: 



Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: --
Detected: 


Detected: 

Detected: 



Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: Story
Detected: 



Detected: 


Detected: 


Detected: 


Detected: 


Detected: 



Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 



Detected: 


Detected: 



Detected: 

Detected: 

Detected: 

Detected: 

Detected: 




Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


71
ORIGINAL SENTENCE
  Zscaler, Inc.
PREDICTED PARAPHRASE



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

[[' Zscaler, Inc.', ' Zscaler, Inc.', ' Zscaler, Inc.', ' Zscaler, Inc.', ' Zscaler, Inc.', ' Zscaler, Inc.', ' Zscaler, Inc.', ' Zscaler, Inc.', ' Zscaler, Inc.', ' Zscaler, Inc.']]


ORIGINAL SENTENCE
  Third Quarter Highlights.
PREDICTED PARAPHRASE



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

[[' Third Quarter Highlights.', ' Third Quarter Highlights.', ' Third Quarter Highlights.', ' Third Quarter Highlights.', ' Third Quarter Highlights.', ' Third Quarter Highlights.', ' Third Quarter Highlights.', ' Third Quarter Highlights.', ' Third Quarter Highlights.', ' Third Quarter Highlights.']]


ORIGINAL SENTENCE
  Revenue grows 63% year-over-year to $286.8 million.
PREDICTED PARAPHRASE



...

[[" Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022.", " Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022.", " Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022.", " Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022.", " Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022.", " Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022.", " Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022.", " Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022.", " Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022.", " Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022."]]


ORIGINAL SENTENCE
  4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.
PREDICTED PARAPHRASE



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

[[' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.', ' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.', ' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.', ' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.', ' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.', ' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.', ' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.', ' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.', ' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.', ' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.']]


ORIGINAL SENTENCE
  Sessions which offer a webcast will be available on the Investor Relations section of the Zscaler website at https://ir.zscaler.com.
PREDICTED PARAPHRASE



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

[[' Sessions which offer a webcast will be available on the Investor Relations section of the Zscaler website at https://ir.zscaler.com.', ' Sessions which offer a webcast will be available on the Investor Relations section of the Zscaler website at https://ir.zscaler.com.', ' Sessions which offer a webcast will be available on the Investor Relations section of the Zscaler website at https://ir.zscaler.com.', ' Sessions which offer a webcast will be available on the Investor Relations section of the Zscaler website at https://ir.zscaler.com.', ' Sessions which offer a webcast will be available on the Investor Relations section of the Zscaler website at https://ir.zscaler.com.', ' Sessions which offer a webcast will be available on the Investor Relations section of the Zscaler website at https://ir.zscaler.com.', ' Sessions which offer a webcast will be available on the Investor Relations section of the Zscaler website at https://ir.zscaler.com.', ' Sessions which offer a webcast will b

In [17]:
createTrainingData(sptrainDir, model, writetofile=True)
writeTrainingData(trainSPFile, sptrainDir)

Input file ../../Summary/DATA/SENTPAIR/Train\APPN_2022-11-03_EP_YH.txt
Input file APPN_2022-11-03_EP_YH
../../Summary/DATA/SENTPAIR/Train
SP file ../../Summary/DATA/SENTPAIR/Train/APPN_2022-11-03_EP_YH_sp.tsv
SP file ..\..\Summary\DATA\SENTPAIR\Train\APPN_2022-11-03_EP_YH_sp.tsv already exists
Input file ../../Summary/DATA/SENTPAIR/Train\BILL_2022-11-03_EP_YH.txt
Input file BILL_2022-11-03_EP_YH
../../Summary/DATA/SENTPAIR/Train
SP file ../../Summary/DATA/SENTPAIR/Train/BILL_2022-11-03_EP_YH_sp.tsv
SP file ..\..\Summary\DATA\SENTPAIR\Train\BILL_2022-11-03_EP_YH_sp.tsv already exists
Input file ../../Summary/DATA/SENTPAIR/Train\CRWD_2022-06-02_EP_YH.txt
Input file CRWD_2022-06-02_EP_YH
../../Summary/DATA/SENTPAIR/Train
SP file ../../Summary/DATA/SENTPAIR/Train/CRWD_2022-06-02_EP_YH_sp.tsv
SP file ..\..\Summary\DATA\SENTPAIR\Train\CRWD_2022-06-02_EP_YH_sp.tsv already exists
Input file ../../Summary/DATA/SENTPAIR/Train\CRWD_2023-05-31_EP_YH.txt
Input file CRWD_2023-05-31_EP_YH
../../Summa

In [18]:
createTrainingData(spdevDir, model, writetofile=True)
writeTrainingData(devSPFile, spdevDir)

Input file ../../Summary/DATA/SENTPAIR/Dev\BILL_2023-02-02_EP_YH.txt
Input file BILL_2023-02-02_EP_YH
../../Summary/DATA/SENTPAIR/Dev
SP file ../../Summary/DATA/SENTPAIR/Dev/BILL_2023-02-02_EP_YH_sp.tsv
SP file ..\..\Summary\DATA\SENTPAIR\Dev\BILL_2023-02-02_EP_YH_sp.tsv already exists
Input file ../../Summary/DATA/SENTPAIR/Dev\CFLT_2023-01-30_EP_YH.txt
Input file CFLT_2023-01-30_EP_YH
../../Summary/DATA/SENTPAIR/Dev
SP file ../../Summary/DATA/SENTPAIR/Dev/CFLT_2023-01-30_EP_YH_sp.tsv
SP file ..\..\Summary\DATA\SENTPAIR\Dev\CFLT_2023-01-30_EP_YH_sp.tsv already exists
Input file ../../Summary/DATA/SENTPAIR/Dev\CRWD_2022-11-29_EP_YH.txt
Input file CRWD_2022-11-29_EP_YH
../../Summary/DATA/SENTPAIR/Dev
SP file ../../Summary/DATA/SENTPAIR/Dev/CRWD_2022-11-29_EP_YH_sp.tsv
SP file ..\..\Summary\DATA\SENTPAIR\Dev\CRWD_2022-11-29_EP_YH_sp.tsv already exists
Input file ../../Summary/DATA/SENTPAIR/Dev\DDOG_2023-05-04_EP_YH.txt
Input file DDOG_2023-05-04_EP_YH
../../Summary/DATA/SENTPAIR/Dev
SP fi

In [19]:
train_df = pd.read_csv(trainSPFile, sep="\t", encoding = "utf-8").astype(str)
eval_df = pd.read_csv(devSPFile, sep="\t", encoding = "utf-8").astype(str)

train_df = train_df.rename(
    columns={"Sentence1": "text_a", "Sentence2": "text_b", "Target": "labels"}
)
eval_df = eval_df.rename(
    columns={"Sentence1": "text_a", "Sentence2": "text_b", "Target": "labels"}
)

train_df = train_df[["text_a", "text_b", "labels"]]
eval_df = eval_df[["text_a", "text_b", "labels"]]

#train_df["prefix"] = "sentpair"
#train_df = train_df[["Sentence1", "Sentence2", "Target"]]

#eval_df["prefix"] = "sentpair"
#eval_df = eval_df[["Sentence1", "Sentence2", "Target"]]

train_df = train_df.dropna()
train_df = train_df[train_df['text_a'].notna()]

eval_df = eval_df.dropna()
eval_df = eval_df[eval_df['text_a'].notna()]

train_df["text_a"] = train_df["text_a"].apply(clean_unnecessary_spaces)
train_df["text_b"] = train_df["text_b"].apply(clean_unnecessary_spaces)
train_df["labels"] = train_df["labels"].astype(int)
print("TRAIN DATA ..............")
print(train_df)

eval_df["text_a"] = eval_df["text_a"].apply(clean_unnecessary_spaces)
eval_df["text_b"] = eval_df["text_b"].apply(clean_unnecessary_spaces)
eval_df["labels"] = eval_df["labels"].astype(int)
print("EVAL DATA ..............")
print(eval_df)
#print(train_df["labels"].value_counts())

TRAIN DATA ..............
                                                 text_a  \
0                                   Appian Corporation.   
1      Third quarter cloud subscription revenue incr...   
2           MCLEAN, Va., Nov. 03, 2022 (GLOBE NEWSWIRE)   
3     -- Appian (Nasdaq: APPN) today announced finan...   
4             Third Quarter 2022 Financial Highlights:.   
...                                                 ...   
1852  Full fiscal year non-GAAP diluted EPS is expec...   
1853  Full fiscal year non-GAAP diluted EPS is expec...   
1854  Full fiscal year non-GAAP diluted EPS is expec...   
1855  Full fiscal year non-GAAP diluted EPS is expec...   
1856  Full fiscal year non-GAAP diluted EPS is expec...   

                                                 text_b  labels  
0                                   Appian Corporation.       1  
1      Third quarter cloud subscription revenue is $...       1  
2           MCLEAN, Va., Nov. 03, 2022 (GLOBE NEWSWIRE)       1  
3

In [15]:
spmodel_args = ClassificationArgs()
spmodel_args.do_sample = True
spmodel_args.train_batch_size = 8
spmodel_args.use_multiprocessing = False
spmodel_args.num_train_epochs = 10
#spmodel_args.learning_rate = 5e-5
#spmodel_args.no_save = True

spmodel_args.eval_batch_size = 8
spmodel_args.evaluate_generated_text = True
spmodel_args.evaluate_during_training = True
spmodel_args.evaluate_during_training_verbose = True
spmodel_args.evaluate_during_training_steps = 50

spmodel_args.overwrite_output_dir = True
spmodel_args.reprocess_input_data = True
spmodel_args.save_eval_checkpoints = False
spmodel_args.save_model_every_epoch = False
spmodel_args.save_steps = -1

spmodel_args.max_length = 64
#spmodel_args.max_seq_length = 512
#spmodel_args.num_return_sequences = 10
#spmodel_args.top_k = 50
#spmodel_args.top_p = 0.95
spmodel_args.fp16 = False
#spmodel_args.num_beams = None

spmodel_args.use_early_stopping = True
spmodel_args.early_stopping_delta = 0.01
spmodel_args.early_stopping_metric = "mcc"
spmodel_args.early_stopping_metric_minimize = False
spmodel_args.early_stopping_patience = 10
#spmodel_args.wandb_project =  "visualization-demo"

spmodel_args.output_dir = "spoutputs/"
#spmodel_args.lazy_loading = True

In [14]:
spmodel = ClassificationModel(
    "roberta", 
    "roberta-base",
    args=spmodel_args,
    use_cuda=True,
)

#spmodel = ClassificationModel(
#    "bert", 
#    "bert-base-uncased",
#    args=spmodel_args,
#    use_cuda=True,
#)

#model = T5Model("t5", "t5-base", args=model_args, use_cuda=True)

# Train the model
spmodel.train_model(
    train_df, eval_df=eval_df, acc=sklearn.metrics.accuracy_score
)
results = spmodel.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_2_3


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/229 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/595 [00:00<?, ?it/s]

...

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_3
INFO:simpletransformers.classification.classification_model:{'mcc': 0.7533902922049743, 'tp': 323, 'tn': 200, 'fp': 53, 'fn': 19, 'auroc': 0.9112983380717934, 'auprc': 0.9200435077716504, 'acc': 0.8789915966386554, 'eval_loss': 0.6696472693715866}
INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to spoutputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/595 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_3


Running Evaluation:   0%|          | 0/75 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.7533902922049743, 'tp': 323, 'tn': 200, 'fp': 53, 'fn': 19, 'auroc': 0.9112983380717934, 'auprc': 0.9200435077716504, 'acc': 0.8789915966386554, 'eval_loss': 0.6696472693715866}


In [16]:
spmodel = ClassificationModel(
    "roberta", "spoutputs", args=spmodel_args
)

predictions, raw_outputs = spmodel.predict(
    [
        [
            "Total revenue was $260.0 million, an increase of 66% year-over-year",
            "Total revenue is $260.0 million"
        ]
    ]
)
print(predictions)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[1]


In [21]:
# Get paraphrasing with ppmodel and spmodel
files = glob.glob(sptestDir+"/*_EP_YH.txt")
#print(files)
#print(inputDir)
if(len(files) > 0):
    for file in (files):
        print("Input file " + file)
        sentences = getSentences(file, nlp)
        for line in sentences:
            #print(line)
            nline = predictPhrase(line, model, spmodel)
            print(line, nline)

Input file ../../Summary/DATA/SENTPAIR/Test\PINS_2023-04-27_EP_YH.txt
https://finance.yahoo.com/news/pinterest-announces-first-quarter-2023-200600201.html https://finance.yahoo.com/news/pinterest-announces-first-quarter-2023-200600201.html
GAAP Net Loss Per Share is $(0.31) in first quarter 2023 @@@ GAAP Net Loss Per Share is $(0.31) in first quarter 2023 @@@
Cash And Cash Equivalents is $1651242 T in first quarter 2023 @@@ Cash And Cash Equivalents is $1651242 T in first quarter 2023 @@@
GAAP Gross Profit is $432MN in first quarter 2023 @@@ GAAP Gross Profit is $432MN in first quarter 2023 @@@
GAAP Gross Margin is 71.63% in first quarter 2023 @@@ GAAP Gross Margin is 71.63% in first quarter 2023 @@@
GAAP Free Cash Flow is $181MN in first quarter 2023 @@@ GAAP Free Cash Flow is $181MN in first quarter 2023 @@@
GAAP Free Cash Flow Margin is 30.11% in first quarter 2023 @@@ GAAP Free Cash Flow Margin is 30.11% in first quarter 2023 @@@


ORIGINAL LINE:
 PG*** SAN FRANCISCO, April 27, 202

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


PREDICTED PARAPHRASE:
 ['SAN FRANCISCO, April 27, 2023--(BUSINESS WIRE)--Pinterest, Inc. (NYSE: PINS) today announced financial results for the quarter ended March 31, 20 23.']


  0%|          | 0/1 [00:00<?, ?it/s]

...

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


PREDICTED PARAPHRASE:
 ['However, it is important  note that material changes  reconciling items could have a significant effect on future GAAP results and, as such, we also believe that any reconciliations provided would imply a degree of precision that could be confusing or misleading  investors..']


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

PREDICTED SENTENCE PAIR:
 [['However, it is important  note that material changes  reconciling items could have a significant effect on future GAAP results and, as such, we also believe that any reconciliations provided would imply a degree of precision that could be confusing or misleading  investors. .', 'However, it is important  note that material changes  reconciling items could have a significant effect on future GAAP results and, as such, we also believe that any reconciliations provided would imply a degree of precision that could be confusing or misleading  investors..']] [1]
PREDICTED SENTENCE:
 However, it is important  note that material changes  reconciling items could have a significant effect on future GAAP results and, as such, we also believe that any reconciliations provided would imply a degree of precision that could be confusing or misleading  investors..
However, it is important  note that material changes  reconciling items could have a significant effect on futu

In [15]:
# Create Para Phrasing training data using ppModel and spModel
ppDataDir = "../../Summary/Refined/PLTR/"
ppTrainDir = "../../Summary/DATA/PARAPHRASE/Train"
ppDevDir = "../../Summary/DATA/PARAPHRASE/Dev"
files = glob.glob(ppDataDir+"/*_EP_YH.txt")
#print(files)
#print(inputDir)
if(len(files) > 0):
    for file in (files):
        print("Input file " + file)
        basefile = os.path.basename(file)
        inputfile = os.path.splitext(basefile)[0]
        inputfile = inputfile+"_phrase.tsv"
        inputfilePath = ppTrainDir+"/"+inputfile
        inputfileDevPath = ppDevDir+"/"+inputfile
        print(inputfilePath, inputfileDevPath)
        outfile = Path(inputfilePath)
        outdevfile = Path(inputfileDevPath)
        if outfile.is_file():
            print(inputfilePath + " Already exists")
        elif outdevfile.is_file():
            print(inputfileDevPath + " Already exists")
        else:
            print("Creating training file {} for paraphrasing".format(inputfilePath))
            with open(outfile, "w", encoding = "utf-8") as of:
                of.write("filename\tSentence1\tSentence2\n")
                sentences = getSentences(file, nlp)
                for line in sentences:
                    #print(line)
                    pline, isProcess = preProcessSent(line)
                    if not isProcess:
                        continue
                    nline = predictPhrase(line, model, spmodel)
                    of.write(inputfile+"\t"+pline+"\t"+nline+"\n")
                    print(pline, nline)
        #break

Input file ../../Summary/Refined/PLTR\PLTR_2022-05-09_EP_YH.txt
../../Summary/DATA/PARAPHRASE/Train/PLTR_2022-05-09_EP_YH_phrase.tsv ../../Summary/DATA/PARAPHRASE/Dev/PLTR_2022-05-09_EP_YH_phrase.tsv
Creating training file ../../Summary/DATA/PARAPHRASE/Train/PLTR_2022-05-09_EP_YH_phrase.tsv for paraphrasing
43


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

 An investor presentation, including supplemental financial information and reconciliations of certain non-GAAP measures to their nearest comparable GAAP measures, will be available through Palantir’s Investor Relations website at https://investors.palantir.com.  An investor presentation, including supplemental financial information and reconciliations of certain non-GAAP measures to their nearest comparable GAAP measures, will be available through Palantir’s Investor Relations website at https://investors.palantir.com.
Input file ../../Summary/Refined/PLTR\PLTR_2023-05-08_EP_YH.txt
../../Summary/DATA/PARAPHRASE/Train/PLTR_2023-05-08_EP_YH_phrase.tsv ../../Summary/DATA/PARAPHRASE/Dev/PLTR_2023-05-08_EP_YH_phrase.tsv
../../Summary/DATA/PARAPHRASE/Train/PLTR_2023-05-08_EP_YH_phrase.tsv Already exists


In [17]:
# Translate proccessed ER text data to new paraphrase with the help of pmodel and spmodel
#origFile = "../../Summary/Refined/APPN/APPN_2022-11-03_EP_YH.txt"
postRefDir = "../../Summary/PostRefined/"
logDir = "../../Summary/Log/"
splTag = ["SCHQ***", "SCBQ***", "SCBF***", "SCHF***", "SCG***", "GF***", "GQ***", "SC***"]

def createPostRefinedER(origFile, nlp, pmodel, spmodel):
    basefile = os.path.basename(origFile)
    #inputfile = os.path.splitext(basefile)[0]
    csym = basefile.split("_")[0]
    #print(csym)
    pRefDir = postRefDir + "/" + csym
    lDir = logDir + "/" + csym
    if not os.path.exists(pRefDir):
        os.makedirs(pRefDir)
    if not os.path.exists(lDir):
        os.makedirs(lDir)
    outfilePath = pRefDir + "/" + basefile
    print(outfilePath)
    logPath = lDir + "/" + basefile
    print(logPath)

    outfile = Path(outfilePath)
    logfile = Path(logPath)

    if outfile.is_file():
        print("Post Refined Para Phrase file " + str(outfile) + " already exists")
    else:
        with open(outfile, "w", encoding = "utf-8") as of:
            f = open(origFile, 'r', encoding="utf-8")
            fl = open(logfile, 'w', encoding="utf-8") # For writing prediction in log file - helps generating more training data

            line = f.readline()
            while line:
                if("ED***" in line):
                    break
                elif("PG***" in line or "NOPAD***" in line or ("***" not in line and "@@@" not in line)):
                    nline = None
                    sentences = getSentences(None, nlp, line)
                    for l in sentences:
                        #print(l)
                        pline, isProcess = preProcessSent(l)
                        if not isProcess:
                            continue
                        if not nline:
                            if("PG***" in line):
                                nline = "PG*** "
                            elif("NOPAD***" in line):
                                nline = "NOPAD*** "
                        pp = predictPhrase(l, pmodel, spmodel, fl)
                        if("Invalid Sentence" in pp):
                            continue
                        if(nline):
                            nline = nline + pp + " "
                        else:
                            nline = pp + " "
                    if(nline):
                        nline = (nline[:len(nline)-1])
                        if(nline[len(nline)-1] == "."):
                            nline = nline[:len(nline)-1] + " ."
                        else:
                            nline = nline + " ."
                        print(nline)
                        of.write(nline+"\n\n")
                else:
                    if(line == "\n"):
                        line = f.readline()
                        continue
                    print(line)
                    of.write(line+"\n\n")
                line = f.readline()
            f.close()
            fl.close()

In [18]:
entPath = "../../Summary/entities/"
rPath = "../../Summary/Refined/"
files = glob.glob(entPath+"/*-ENTITIES.json")   
if(len(files) > 0):
    for file in (files):
        basefile = os.path.basename(file)
        filename = os.path.splitext(basefile)[0]
        csym = filename.split("-")[0]
        #print(csym)
        cPath = rPath + "/" + csym
        #print(cPath)
        cfiles = glob.glob(cPath+"/*.txt")
        if(len(cfiles) > 0):
            for cf in cfiles:
                if(isPhrasingRqd(cf)):
                    createPostRefinedER(cf, nlp, model, spmodel)

../../Summary/PostRefined//APPN/APPN_2022-08-04_EP_YH.txt
../../Summary/Log//APPN/APPN_2022-08-04_EP_YH.txt
Post Refined Para Phrase file ..\..\Summary\PostRefined\APPN\APPN_2022-08-04_EP_YH.txt already exists
../../Summary/PostRefined//APPN/APPN_2022-11-03_EP_YH.txt
../../Summary/Log//APPN/APPN_2022-11-03_EP_YH.txt
Post Refined Para Phrase file ..\..\Summary\PostRefined\APPN\APPN_2022-11-03_EP_YH.txt already exists
../../Summary/PostRefined//APPN/APPN_2023-02-16_EP_YH.txt
../../Summary/Log//APPN/APPN_2023-02-16_EP_YH.txt
Post Refined Para Phrase file ..\..\Summary\PostRefined\APPN\APPN_2023-02-16_EP_YH.txt already exists
../../Summary/PostRefined//APPN/APPN_2023-05-09_EP_YH.txt
../../Summary/Log//APPN/APPN_2023-05-09_EP_YH.txt
Post Refined Para Phrase file ..\..\Summary\PostRefined\APPN\APPN_2023-05-09_EP_YH.txt already exists
../../Summary/PostRefined//APPN/APPN_2023-08-03_EP_YH.txt
../../Summary/Log//APPN/APPN_2023-08-03_EP_YH.txt
Post Refined Para Phrase file ..\..\Summary\PostRefi


Cash And Cash Equivalents is $607175 T in second quarter 2024 @@@

GAAP Gross Profit is $318454 T in second quarter 2024 @@@

GAAP Gross Margin is 75.14% in second quarter 2024 @@@

GAAP Free Cash Flow is $(25.9MN) in second quarter 2024 @@@

GAAP Free Cash Flow Margin is (6.12)% in second quarter 2024 @@@



ORIGINAL LINE:
 PG*** Second Quarter Fiscal 2024 Total Revenue of $423.8 million, up 40% Year-over-Year.


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


PREDICTED PARAPHRASE:
 [' Second Quarter Fiscal 2024 Total Revenue is $423.8 million.', 'Second Quarter Fiscal 2024 Total Revenue is $423.8 million.']


...

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


PREDICTED PARAPHRASE:
 ['Accordingly, a reconciliation of non-GAAP gross margin and non-gaAP operating margin is not available without unreasonable effort.', 'Accordingly, a reconciliation of non-GAAP gross margin and non-PAAP operating margin is not available without unreasonable effort.', 'Accordingly, a reconciliation of non-GAAP gross margin and non-AP operating margin is not available without unreasonable effort.', 'Accordingly, a reconciliation of non-GAAP gross margin and non- GAAP operating margin is not available without unreasonable effort.', 'Accordingly, a reconciliation of non-GAAP gross margin and non-DAAP operating margin is not available without unreasonable effort.']


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

PREDICTED SENTENCE PAIR:
 [['Accordingly, a reconciliation of non-GAAP gross margin and non-GAAP operating margin is not available without unreasonable effort.', 'Accordingly, a reconciliation of non-GAAP gross margin and non-gaAP operating margin is not available without unreasonable effort.']] [1]
PREDICTED SENTENCE:
 Accordingly, a reconciliation of non-GAAP gross margin and non-gaAP operating margin is not available without unreasonable effort.
PG*** Guidance for non-GAAP financial measures excludes stock-based compensation expense, employer payroll tax on employee stock transactions, amortization expense of acquired intangible assets, acquisition-related compensation costs, restructuring charges, and gain on strategic investments. We have not provided the most directly comparable GAAP measures because certain items are out of our control or cannot be reasonably predicted. Accordingly, a reconciliation of non-GAAP gross margin and non-gaAP operating margin is not available without 

In [27]:
probFile = "../../Summary/DATA/SENTPAIR/Train/OTHER_EP_YH_sp.tsv"
with open(probFile, "r") as of:
    cnt = 0
    line = of.readline()
    while(line):
        cnt = cnt + 1
        print(cnt, line)
        line = of.readline()

1 filename	Sentence1	Sentence2	Target

2 APPN_2022-11-03_EP_YH	 Appian Corporation.	 Appian Corporation.	1

3 APPN_2022-11-03_EP_YH	Net cash used in operating activities was $(29.7) million for the three months ended June 30, 2022 compared to $(6.6) million of net cash used in operating activities for the same period in 2021.	Net cash used in operating activitiesis $(29.7) million for the three months ended June 30, 2022.	0

4 APPN_2022-11-03_EP_YH	Adjusted EBITDA loss is expected to be between $(15.0) million and $(13.0) million.	Adjusted EBITDA loss is expected to be between $(15.0) million and $(13.0) million.	1

5 APPN_2022-11-03_EP_YH	Adjusted EBITDA loss is expected to be between $(15.0) million and $(13.0) million.	Adjusted EBITDA loss is expected to be between $(15.0) million and $(13.0)\\ million.	0

6 APPN_2022-11-03_EP_YH	Cloud subscription revenue retention rate was 115% as of March 31, 2023.	Cloud subscription revenue retention rate is 115% as of March 31, 2023.	1

7 BILL_